In [1]:
import re
import string
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
df = pd.read_csv("YORUMLAR.csv", encoding='cp1254', header=None, names=["Name", "Brand", "Comment","Rate"])
df.head()

,Name,Brand,Comment,Rate
0,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,oyun için aldık. f/p ürünü,5.0
1,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,Gerçekten aldığım için hiç pişman olmadım hızl...,5.0
2,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok beğendim indirimde almıştım,5.0
3,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok iyi paketlenmiş teşekkürler,5.0
4,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,İndirimde iken aldikk çok iyi cihaz 10/10,5.0


In [3]:
floatize = lambda x: float(x[0:-2]) if isinstance(x, str) else x

df["Rate"] = df["Rate"].apply(floatize)
df["Rate"].value_counts()

5.0    998
1.0    345
4.0    336
3.0    174
2.0    102
Name: Rate, dtype: int64

In [4]:
df.drop(df[df["Rate"] == 3].index, inplace = True)
df["Rate"] = df["Rate"].replace(1, 0)
df["Rate"] = df["Rate"].replace(2, 0)
df["Rate"] = df["Rate"].replace(4, 1)
df["Rate"] = df["Rate"].replace(5, 1)
df["Rate"].value_counts()

1.0    1334
0.0     447
Name: Rate, dtype: int64

In [5]:
df.reset_index(inplace = True)
df.drop("index", axis = 1, inplace = True)
df.head()


,Name,Brand,Comment,Rate
0,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,oyun için aldık. f/p ürünü,1.0
1,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,Gerçekten aldığım için hiç pişman olmadım hızl...,1.0
2,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok beğendim indirimde almıştım,1.0
3,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok iyi paketlenmiş teşekkürler,1.0
4,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,İndirimde iken aldikk çok iyi cihaz 10/10,1.0


In [6]:
df["Comment"] = df["Comment"].fillna("").apply(lambda x: x.lower())
df.head()


,Name,Brand,Comment,Rate
0,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,oyun için aldık. f/p ürünü,1.0
1,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,gerçekten aldığım için hiç pişman olmadım hızl...,1.0
2,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok beğendim indirimde almıştım,1.0
3,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok iyi paketlenmiş teşekkürler,1.0
4,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,i̇ndirimde iken aldikk çok iyi cihaz 10/10,1.0


In [7]:
def remove_punctuation(text):
    no_punc = [words for words in text if words not in string.punctuation]
    word_wo_punc = "".join(no_punc)
    return word_wo_punc

df["Comment"] = df["Comment"].apply(lambda x: remove_punctuation(x))
df["Comment"] = df["Comment"].apply(lambda x: x.replace("\r", " "))
df["Comment"] = df["Comment"].apply(lambda x: x.replace("\n", " "))

df.head()

,Name,Brand,Comment,Rate
0,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,oyun için aldık fp ürünü,1.0
1,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,gerçekten aldığım için hiç pişman olmadım hızl...,1.0
2,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok beğendim indirimde almıştım,1.0
3,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok iyi paketlenmiş teşekkürler,1.0
4,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,i̇ndirimde iken aldikk çok iyi cihaz 1010,1.0


In [8]:
def remove_numeric(corpus):
    output = "".join(words for words in corpus if not words.isdigit())
    return output

df["Comment"] = df["Comment"].apply(lambda x: remove_numeric(x)) 
df.head()

,Name,Brand,Comment,Rate
0,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,oyun için aldık fp ürünü,1.0
1,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,gerçekten aldığım için hiç pişman olmadım hızl...,1.0
2,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok beğendim indirimde almıştım,1.0
3,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,çok iyi paketlenmiş teşekkürler,1.0
4,Monster Abra A7 V13.2.3 Intel Core I5-11400h 1...,MONSTER,i̇ndirimde iken aldikk çok iyi cihaz,1.0


In [9]:
target = df["Rate"].values.tolist()
data = df["Comment"].values.tolist()

cutoff = int(len(data)*0.80)

X_train, X_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [10]:
num_words = 10000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)
# tokenizer.word_index


In [11]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

print([X_train[1000]])
print(X_train_tokens[1000])


['pc gerçekten mükemmel oğluma karne hediyesi olarak aldım bayıldı hiç tereddüt etmeden alabilirsiniz oyunlarıda kasmadan çok rahat oynuyor']
[112, 57, 70, 293, 521, 160, 21, 13, 3395, 58, 319, 413, 134, 2424, 719, 1, 191, 537]


In [12]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([  5,  14,   4, ..., 115,  37,  61])

In [13]:
np.mean(num_tokens)

22.848932676518885

In [14]:
np.max(num_tokens)

249

In [15]:
max_tokens = np.mean(num_tokens) + (2*np.std(num_tokens))
max_tokens = int(max_tokens)
max_tokens

70

In [16]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9594964422550629

In [17]:
X_train_pad = pad_sequences(X_train_tokens, maxlen = max_tokens) 
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_tokens)

print(X_train_pad.shape)
print(X_test_pad.shape)

(1461, 70)
(366, 70)


In [18]:
np.array(X_train_tokens[800])

array([ 109, 1027, 1862,  171,  200])

In [19]:
X_train_pad[500]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        342,    7, 1911,    5])

In [20]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = " ".join(words) # Kelimeler aralarında boşluk bırakılarak ard arda yazılacaktır.
    return text

In [21]:
tokens_to_string(X_train_tokens[350])

'ürünü yaklaşık aydır kullanıyorum ve kesinlikle önermiyorum şarja takınca kendiliğinden kapanıyor önemli bir şey yapıyorsanız geçmiş olsun ara ara ekran gidip geliyor bazen açamıyorsunuz bilgisayarın canı ne zaman isterse o zaman açılıyor fan sesi çok fazla aşırı ısınıyor hoparlörleri çok kötü şimdiye kadar farklı marka bilgisayar kullandım kesinlikle aralarında en kötüsü'

In [22]:
embedding_size = 50
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = embedding_size, input_length = max_tokens, name = "embedding_layer"))
model.add(GRU(units = 16, return_sequences = True))
model.add(GRU(units = 8, return_sequences = True))
model.add(GRU(units = 4))
model.add(Dense(1, activation = "sigmoid"))

In [23]:
optimizer = Adam(lr = 1e-3)
model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 70, 50)           500000    
                                                                 
 gru (GRU)                   (None, 70, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 70, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [24]:
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

model.fit(X_train_pad, y_train, epochs = 5, batch_size = 256)

Epoch 1/5
6/6 [==============================] - 5s 85ms/step - loss: nan - accuracy: 0.2560
Epoch 2/5
6/6 [==============================] - 0s 76ms/step - loss: nan - accuracy: 0.2341
Epoch 3/5
6/6 [==============================] - 0s 75ms/step - loss: nan - accuracy: 0.2341
Epoch 4/5
6/6 [==============================] - 0s 78ms/step - loss: nan - accuracy: 0.2341
Epoch 5/5
6/6 [==============================] - 0s 81ms/step - loss: nan - accuracy: 0.2341


In [25]:
y_pred = model.predict(X_test_pad[0:1000])
y_pred = y_pred.T[0]
# y_pred

12/12 [==============================] - 1s 10ms/step


In [26]:
cls_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])
cls_true = np.array(y_test[0:1000])

In [27]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]
# incorrect

In [28]:
len(incorrect)

261

In [29]:
idx = incorrect[10]
X_test[idx]

'çok ısınıyor fakat iyi bir bilgisayar'

In [30]:
y_pred[idx]

nan

In [31]:
cls_true[idx]

1.0